In [1]:
import pandas as pd
import numpy as np
import scipy as sc


import warnings 
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
%pylab inline

import datetime
import dateutil.relativedelta

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from tqdm import tqdm_notebook
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import precision_score, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer as DV

In [ ]:
model = pd.read_excel('model_data.xlsx')

In [ ]:
# Строим целевые переменные 

model['y_3'] = model['shifted_3'] - model['shifted_3_sec']
model['y_3'].where(model['y_3']<0, 1, inplace = True)
model['y_3'].where(model['y_3']>0, 0, inplace = True)


model['y_6'] = model['shifted_6'] - model['shifted_6_sec']
model['y_6'].where(model['y_6']<0, 1, inplace = True)
model['y_6'].where(model['y_6']>0, 0, inplace = True)


model['y_12'] = model['shifted_12'] - model['shifted_12_sec']
model['y_12'].where(model['y_12']<0, 1, inplace = True)
model['y_12'].where(model['y_12']>0, 0, inplace = True)

In [ ]:
model.reset_index(inplace = True, drop = True)

In [ ]:
spisok = ['0',   '1',   '2',   '5',   '6',   '7',   '9',   '14',   '15',
   '17',   '21',   '22',   '23',   '24',   '25',   '26',   '31',   '32',   '33',
   '38',   '39',   '40',   '41',   '42',   '43',   '44',   '45',   '47',   '48',   '49']
spisok = map(int, spisok)
l = list(spisok)

In [ ]:
# Для 17 года
model_train_2017 = model.iloc[:4545, :]
model_test_2017 = model.iloc[4545:, :]

# Для 16 года
model_train_2016 = model.iloc[:4200, :]
model_test_2016 = model.iloc[4200:4445, :]

model_train_2017 = model_train_2017.sample(frac=1, random_state = 1)
model_train_2016 = model_train_2016.sample(frac=1, random_state = 1)

Y_train_2017 = model_train_2017.iloc[:, -12:]
Y_test_2017 = model_test_2017.iloc[:, -12:]
Y_6_train_2017 = Y_train_2017.iloc[:, -2:-1]
Y_6_test_2017 = Y_test_2017.iloc[:, -2:-1]

Y_train_2016 = model_train_2016.iloc[:, -12:]
Y_test_2016 = model_test_2016.iloc[:, -12:]
Y_6_train_2016 = Y_train_2016.iloc[:, -2:-1]
Y_6_test_2016 = Y_test_2016.iloc[:, -2:-1]

X_train_2017 = model_train_2017.iloc[:, :-12]
X_test_2017 = model_test_2017.iloc[:, :-12]

X_train_2016 = model_train_2016.iloc[:, :-12]
X_test_2016 = model_test_2016.iloc[:, :-12]

features_num_2017 = []
features_cat_2017 = []
for feature in X_test_2017.columns:
    if (X_test_2017[feature].dtype == 'float64') or (X_test_2017[feature].dtype == 'int64'):
        features_num_2017.append(feature)
    else:
        features_cat_2017.append(feature)
        
features_num_2016 = []
features_cat_2016 = []
for feature in X_test_2016.columns:
    if (X_test_2016[feature].dtype == 'float64') or (X_test_2016[feature].dtype == 'int64'):
        features_num_2016.append(feature)
    else:
        features_cat_2016.append(feature)

comp_train_2017 = X_train_2017[features_cat_2017[:4]]
comp_test_2017 = X_test_2017[features_cat_2017[:4]]

comp_train_2016 = X_train_2016[features_cat_2016[:4]]
comp_test_2016 = X_test_2016[features_cat_2016[:4]]

comp_train_2017['year'] = pd.DatetimeIndex(comp_train_2017['DataDate']).year
comp_test_2017['year'] = pd.DatetimeIndex(comp_test_2017['DataDate']).year

comp_train_2016['year'] = pd.DatetimeIndex(comp_train_2016['DataDate']).year
comp_test_2016['year'] = pd.DatetimeIndex(comp_test_2016['DataDate']).year

X_train_num_2017 = X_train_2017[features_num_2017]
X_test_num_2017 = X_test_2017[features_num_2017]

X_train_num_2016 = X_train_2016[features_num_2016]
X_test_num_2016 = X_test_2016[features_num_2016]

'''1. Кодируем категориальные признаки 2017'''

X_sec_2017 = model[features_cat_2017[3:]]
label_encoder_2017 = preprocessing.LabelEncoder() 
X_sec_enc_2017 = label_encoder_2017.fit_transform(X_sec_2017) 

X_sec_enc_train_2017 = X_sec_enc_2017[:X_train_num_2017.shape[0]]
X_sec_enc_test_2017 = X_sec_enc_2017[X_train_num_2017.shape[0]:X_train_num_2017.shape[0]+X_test_num_2017.shape[0]]

'''1. Кодируем категориальные признаки _2016'''

X_sec_2016 = model[features_cat_2016[3:]]
label_encoder_2016 = preprocessing.LabelEncoder() 
X_sec_enc_2016 = label_encoder_2016.fit_transform(X_sec_2016) 

X_sec_enc_train_2016 = X_sec_enc_2016[:X_train_num_2016.shape[0]]
X_sec_enc_test_2016 = X_sec_enc_2016[X_train_num_2016.shape[0]:X_train_num_2016.shape[0]+X_test_num_2016.shape[0]]

'''3. Масштабируем реальные признаки _2017'''

scaler_2017 = StandardScaler()
X_num_train_scaled_2017 = scaler_2017.fit_transform(X_train_num_2017)
X_num_test_scaled_2017 = scaler_2017.transform(X_test_num_2017)

'''3. Масштабируем реальные признакиь _2016'''

scaler_2016 = StandardScaler()
X_num_train_scaled_2016 = scaler_2016.fit_transform(X_train_num_2016)
X_num_test_scaled_2016 = scaler_2016.transform(X_test_num_2016)

"""4. Стакаем категориальные и реальные фичи _2017"""

train_sample_2017 = np.hstack((X_num_train_scaled_2017, X_sec_enc_train_2017.reshape(-1, 1)))
test_sample_2017 = np.hstack((X_num_test_scaled_2017, X_sec_enc_test_2017.reshape(-1, 1)))

"""4. Стакаем категориальные и реальные фичи _2016"""

train_sample_2016 = np.hstack((X_num_train_scaled_2016, X_sec_enc_train_2016.reshape(-1, 1)))
test_sample_2016 = np.hstack((X_num_test_scaled_2016, X_sec_enc_test_2016.reshape(-1, 1)))

In [ ]:

'''1. Кодируем категориальные признаки lin'''
X_sec_2017 = model[features_cat_2017[3:]]
encoder_2017 = DV(sparse = False)
X_sec_enc_2017_lin = encoder_2017.fit_transform(X_sec_2017.T.to_dict().values())

X_sec_2016 = model[features_cat_2016[3:]]
encoder_2016 = DV(sparse = False)
X_sec_enc_2016_lin = encoder_2016.fit_transform(X_sec_2016.T.to_dict().values())

X_sec_enc_train_2017_lin = X_sec_enc_2017_lin[:X_train_num_2017.shape[0]]
X_sec_enc_test_2017_lin = X_sec_enc_2017_lin[X_train_num_2017.shape[0]:X_train_num_2017.shape[0]+X_test_num_2017.shape[0]]

X_sec_enc_train_2016_lin = X_sec_enc_2016_lin[:X_train_num_2016.shape[0]]
X_sec_enc_test_2016_lin = X_sec_enc_2016_lin[X_train_num_2016.shape[0]:X_train_num_2016.shape[0]+X_test_num_2016.shape[0]]

"""4. Стакаем категориальные и реальные фичи lin"""
X_num_train_scaled_2016 = X_num_train_scaled_2016.T[l[:-1]].T
X_num_test_scaled_2016 = X_num_test_scaled_2016.T[l[:-1]].T

train_sample_lin_2016 = np.hstack((X_num_train_scaled_2016, X_sec_enc_train_2016_lin))
test_sample_lin_2016 = np.hstack((X_num_test_scaled_2016, X_sec_enc_test_2016_lin))

"""4. Стакаем категориальные и реальные фичи lin"""

X_num_train_scaled_2017 = X_num_train_scaled_2017.T[l[:-1]].T
X_num_test_scaled_2017 = X_num_test_scaled_2017.T[l[:-1]].T

train_sample_lin_2017 = np.hstack((X_num_train_scaled_2017, X_sec_enc_train_2017_lin))
test_sample_lin_2017 = np.hstack((X_num_test_scaled_2017, X_sec_enc_test_2017_lin))

In [ ]:
train_sample_2017 = train_sample_2017.T[l].T
test_sample_2017 = test_sample_2017.T[l].T

train_sample_2016 = train_sample_2016.T[l].T
test_sample_2016 = test_sample_2016.T[l].T

In [ ]:
best_lin_6m_2017 = LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=123, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)


best_lin_6m_2016 = LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=123, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

best_lin_6m_2017.fit(train_sample_lin_2017, Y_6_train_2017)
best_lin_6m_2016.fit(train_sample_lin_2016, Y_6_train_2016)

In [ ]:
# Деревення модель _2017
best_rfc_6m_2017 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features=6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=13, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=35, n_jobs=-1,
                       oob_score=True, random_state=123, verbose=0,
                       warm_start=False)
best_rfc_6m_2017.fit(train_sample_2017, Y_6_train_2017)

# Деревення модель _2016
best_rfc_6m_2016 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features=6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=13, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=35, n_jobs=-1,
                       oob_score=True, random_state=123, verbose=0,
                       warm_start=False)
best_rfc_6m_2016.fit(train_sample_2016, Y_6_train_2016)

In [ ]:
# модель XGBoost _2017
best_xgb_6m_opt_2017 = xgb.XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=9, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, silent=1, subsample=0.6,
              tree_method=None, validate_parameters=False, verbosity=None)
best_xgb_6m_opt_2017.fit(train_sample_2017, Y_6_train_2017)

# модель XGBoost _2016
best_xgb_6m_opt_2016 = xgb.XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=9, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, silent=1, subsample=0.6,
              tree_method=None, validate_parameters=False, verbosity=None)
best_xgb_6m_opt_2016.fit(train_sample_2016, Y_6_train_2016)

In [ ]:
# Инициализируем страифицированную разбивку нашего датасета для валидации _2017
skf = StratifiedKFold(n_splits = 5, random_state = 123)

rfc_12_2017 = RandomForestClassifier(random_state = 123, n_jobs = -1, oob_score = True)

results_12_2017 = cross_val_score(rfc_12_2017, train_sample_2017, Y_6_train_2017, cv=skf)

# Оцениваем долю верных ответов на тестовом датасете
print("CV accuracy score model_6m_2017: {:.2f}%".format(results_12_2017.mean()*100))


# Инициализируем страифицированную разбивку нашего датасета для валидации _2016
skf = StratifiedKFold(n_splits = 5, random_state = 123)

rfc_12_2016 = RandomForestClassifier(random_state = 123, n_jobs = -1, oob_score = True)

results_12_2016 = cross_val_score(rfc_12_2016, train_sample_2016, Y_6_train_2016, cv=skf)

# Оцениваем долю верных ответов на тестовом датасете
print("CV accuracy score model_6m_2016: {:.2f}%".format(results_12_2016.mean()*100))


In [ ]:
# LR 2017
proba_l_2017 = best_lin_6m_2017.predict_proba(test_sample_lin_2017)
auc_l_2017 = roc_auc_score(np.array(Y_6_test_2017), np.transpose(proba_l_2017)[1])
print('auc_l_2017 {:.5}'.format(auc_l_2017), '\n')
fpr_l_2017, tpr_l_2017, _ = roc_curve(Y_6_test_2017, proba_l_2017[:,1])

# LR 2016
proba_l_2016 = best_lin_6m_2016.predict_proba(test_sample_lin_2016)
auc_l_2016 = roc_auc_score(np.array(Y_6_test_2016), np.transpose(proba_l_2016)[1])
print('auc_l_2016 {:.5}'.format(auc_l_2016), '\n')
fpr_l_2016, tpr_l_2016, _ = roc_curve(Y_6_test_2016, proba_l_2016[:,1])

# Forest _2017
proba_F_2017 = best_rfc_6m_2017.predict_proba(test_sample_2017)
auc_F_2017 = roc_auc_score(np.array(Y_6_test_2017), np.transpose(proba_F_2017)[1])
print('auc_F_2017 {:.5}'.format(auc_F_2017), '\n')
fpr_F_2017, tpr_F_2017, _ = roc_curve(Y_6_test_2017, proba_F_2017[:,1])

# Forest _2016
proba_F_2016 = best_rfc_6m_2016.predict_proba(test_sample_2016)
auc_F_2016 = roc_auc_score(np.array(Y_6_test_2016), np.transpose(proba_F_2016)[1])
print('auc_F_2016 {:.5}'.format(auc_F_2016), '\n')
fpr_F_2016, tpr_F_2016, _ = roc_curve(Y_6_test_2016, proba_F_2016[:,1])

# Boost _2017
proba_B_2017 = best_xgb_6m_opt_2017.predict_proba(test_sample_2017)
auc_B_2017 = roc_auc_score(np.array(Y_6_test_2017), np.transpose(proba_B_2017)[1])
print('auc_B_2017 {:.5}'.format(auc_B_2017), '\n')
fpr_B_2017, tpr_B_2017, _ = roc_curve(Y_6_test_2017, proba_B_2017[:,1])

# Boost _2016
proba_B_2016 = best_xgb_6m_opt_2016.predict_proba(test_sample_2016)
auc_B_2016 = roc_auc_score(np.array(Y_6_test_2016), np.transpose(proba_B_2016)[1])
print('auc_B_2016 {:.5}'.format(auc_B_2016), '\n')
fpr_B_2016, tpr_B_2016, _ = roc_curve(Y_6_test_2016, proba_B_2016[:,1])

In [ ]:
# _2017

fig = plt.figure(figsize=(35, 10))
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
ax13 = fig.add_subplot(131);ax14 = fig.add_subplot(132); ax15 = fig.add_subplot(133);

ax13.plot(fpr_F_2017, tpr_F_2017, label = 'Forest_2018')
ax13.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax13.set_xlim([-0.05, 1.05])
ax13.set_ylim([-0.05, 1.05])
ax13.set_xlabel('False Positive Rate')
ax13.set_ylabel('True Positive Rate')
ax13.set_title('ROC curve_2018 (AUC_ROC = 0.62)')
ax13.legend(loc = "lower right")

ax14.plot(fpr_B_2017, tpr_B_2017, label = 'Boost_2018')
ax14.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax14.set_xlim([-0.05, 1.05])
ax14.set_ylim([-0.05, 1.05])
ax14.set_xlabel('False Positive Rate')
ax14.set_ylabel('True Positive Rate')
ax14.set_title('ROC curve_2018 (AUC_ROC = 0.56)')
ax14.legend(loc = "lower right")

ax15.plot(fpr_l_2017, tpr_l_2017, label = 'LR_2018')
ax15.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax15.set_xlim([-0.05, 1.05])
ax15.set_ylim([-0.05, 1.05])
ax15.set_xlabel('False Positive Rate')
ax15.set_ylabel('True Positive Rate')
ax15.set_title('ROC curve_2018 (AUC_ROC = 0.545)')
ax15.legend(loc = "lower right")

In [ ]:
# _2016

fig = plt.figure(figsize=(35, 10))
ax13 = fig.add_subplot(131);ax14 = fig.add_subplot(132); ax15 = fig.add_subplot(133);
sns.set(style="whitegrid", color_codes=True, font_scale = 2)

ax13.plot(fpr_F_2016, tpr_F_2016, label = 'Forest_2017')
ax13.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax13.set_xlim([-0.05, 1.05])
ax13.set_ylim([-0.05, 1.05])
ax13.set_xlabel('False Positive Rate')
ax13.set_ylabel('True Positive Rate')
ax13.set_title('ROC curve_2017 (AUC_ROC = 0.545)')
ax13.legend(loc = "lower right")

ax14.plot(fpr_B_2016, tpr_B_2016, label = 'Boost_2017')
ax14.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax14.set_xlim([-0.05, 1.05])
ax14.set_ylim([-0.05, 1.05])
ax14.set_xlabel('False Positive Rate')
ax14.set_ylabel('True Positive Rate')
ax14.set_title('ROC curve_2017 (AUC_ROC = 0.55)')
ax14.legend(loc = "lower right")

ax15.plot(fpr_l_2016, tpr_l_2016, label = 'LR_2017')
ax15.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax15.set_xlim([-0.05, 1.05])
ax15.set_ylim([-0.05, 1.05])
ax15.set_xlabel('False Positive Rate')
ax15.set_ylabel('True Positive Rate')
ax15.set_title('ROC curve_2017 (AUC_ROC = 0.56)')
ax15.legend(loc = "lower right")

In [ ]:
# Mean _2017
proba_F_2017 = best_rfc_6m_2017.predict_proba(test_sample_2017)
proba_B_2017 = best_xgb_6m_opt_2017.predict_proba(test_sample_2017)
proba_l_2017 = best_lin_6m_2017.predict_proba(test_sample_lin_2017)
proba_M_2017 = (proba_F_2017 + proba_B_2017 + proba_l_2017) / 3

auc_M_2017 = roc_auc_score(np.array(Y_6_test_2017), np.transpose(proba_M_2017)[1])
print('auc_M_2017 {:.5}'.format(auc_M_2017), '\n')
fpr_M_2017, tpr_M_2017, _ = roc_curve(Y_6_test_2017, proba_M_2017[:,1])

# Mean _2016
proba_F_2016 = best_rfc_6m_2016.predict_proba(test_sample_2016)
proba_B_2016 = best_xgb_6m_opt_2016.predict_proba(test_sample_2016)
proba_l_2016 = best_lin_6m_2016.predict_proba(test_sample_lin_2016)
proba_M_2016 = (proba_F_2016 + proba_B_2016 + proba_l_2016) / 3

auc_M_2016 = roc_auc_score(np.array(Y_6_test_2016), np.transpose(proba_M_2016)[1])
print('auc_M_2016 {:.5}'.format(auc_M_2016), '\n')
fpr_M_2016, tpr_M_2016, _ = roc_curve(Y_6_test_2016, proba_M_2016[:,1])


In [ ]:
7# _2017

fig = plt.figure(figsize=(32, 11))
ax13 = fig.add_subplot(121);ax14 = fig.add_subplot(122);

ax13.plot(fpr_M_2017, tpr_M_2017, label = 'stack_2018')
ax13.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax13.set_xlim([-0.05, 1.05])
ax13.set_ylim([-0.05, 1.05])
ax13.set_xlabel('False Positive Rate')
ax13.set_ylabel('True Positive Rate')
ax13.set_title('ROC curve_2018 (AUC_ROC = 0.59)')
ax13.legend(loc = "lower right")

# _2016

ax14.plot(fpr_M_2016, tpr_M_2016, label = 'stack_2017')
ax14.plot([0, 1], [0, 1], '--', color = 'grey', label = 'random')
ax14.set_xlim([-0.05, 1.05])
ax14.set_ylim([-0.05, 1.05])
ax14.set_xlabel('False Positive Rate')
ax14.set_ylabel('True Positive Rate')
ax14.set_title('ROC curve_2017 (AUC_ROC = 0.56)')
ax14.legend(loc = "lower right")

In [ ]:
probs_2016 = pd.DataFrame(proba_M_2016, columns = ['0','1'])
comp_test_2016['Prob_1'] =  np.array(probs_2016.iloc[:, 1])

true_2016 = Y_test_2016.loc[:, ['shifted_6', 'shifted_6_sec', 'y_6']]

comp_test_2016['shifted_6'] = np.array(true_2016.iloc[:, 0])
comp_test_2016['shifted_6_sec'] = np.array(true_2016.iloc[:, 1])
comp_test_2016['y_6'] = np.array(true_2016.iloc[:, 2])
comp_test_2016.reset_index(inplace = True, drop = True)
#comp_test_hard_2016 = comp_test_2016[comp_test_2016.Prob_1 >= .6]

In [ ]:
probs_2017 = pd.DataFrame(proba_M_2017, columns = ['0','1'])
comp_test_2017['Prob_1'] =  np.array(probs_2017.iloc[:, 1])

true_2017 = Y_test_2017.loc[:, ['shifted_6', 'shifted_6_sec', 'y_6']]

comp_test_2017['shifted_6'] = np.array(true_2017.iloc[:, 0])
comp_test_2017['shifted_6_sec'] = np.array(true_2017.iloc[:, 1])
comp_test_2017['y_6'] = np.array(true_2017.iloc[:, 2])
comp_test_2017.reset_index(inplace = True, drop = True)
# comp_test_hard_2017 = comp_test_2017[comp_test_2017.Prob_1 >= .55]

In [ ]:
comp_test_2016['rating'] = 0
i = 1
alpha = 0
while alpha <= 0.9:     
    boundaries = np.percentile(comp_test_2016.Prob_1, [100 * alpha, 100 * (alpha + 0.1)])
    comp_test_2016['rating'][(comp_test_2016.Prob_1 >= boundaries[0]) & (comp_test_2016.Prob_1 <= boundaries[1])] = i
    alpha += 0.1
    i += 1

comp_test_2017['rating'] = 0
i = 1
alpha = 0
while alpha <= 0.9:     
    boundaries = np.percentile(comp_test_2017.Prob_1, [100 * alpha, 100 * (alpha + 0.1)])
    comp_test_2017['rating'][(comp_test_2017.Prob_1 >= boundaries[0]) & (comp_test_2017.Prob_1 <= boundaries[1])] = i
    alpha += 0.1
    i += 1

In [ ]:
comp_test_2017.sort_values(by = ['rating', 'Prob_1'], ascending = False, inplace = True)
comp_test_2016.sort_values(by = ['rating', 'Prob_1'], ascending = False, inplace = True)

In [ ]:
top_10_2017 = comp_test_2017[comp_test_2017.rating == 10]
top_10_2017.to_excel('top_10_2017.xlsx')

In [ ]:
pd.Series.value_counts(top_10_2017['Sector'])

In [ ]:
top_10_2016 = comp_test_2016[comp_test_2016.rating == 10]
top_10_2016.to_excel('top_10_2016.xlsx') 

In [ ]:
pd.Series.value_counts(top_10_2016['Sector'])

In [ ]:
prob_sec_mean_2016 = pd.DataFrame()
prob_sec_mean_2016['num'] = ceil((pd.Series.value_counts(comp_test_hard_2016['Sector']) / comp_test_hard_2016['Sector'].count()) * 20)
prob_sec_mean_2016.reset_index(inplace = True)
prob_sec_mean_2016.rename(columns = {'index': 'Sector'}, inplace = True)

In [ ]:
pd.Series.value_counts(itog_2016['Sector'])

In [ ]:
prob_sec_mean_2017 = pd.DataFrame()
prob_sec_mean_2017['num'] = ceil((pd.Series.value_counts(comp_test_hard_2017['Sector']) / comp_test_hard_2017['Sector'].count()) * 20)
prob_sec_mean_2017.reset_index(inplace = True)
prob_sec_mean_2017.rename(columns = {'index': 'Sector'}, inplace = True)

In [ ]:
new_2016 = pd.DataFrame()
for sector in list(set(prob_sec_mean_2016.Sector)):
    l_2016 = comp_test_hard_2016[comp_test_hard_2016.Sector == sector]
    l_2016 = l_2016.sort_values(by = [ 'Prob_1'], ascending = False).head(int(prob_sec_mean_2016['num'][prob_sec_mean_2016.Sector == sector]))
    new_2016 = pd.concat([new_2016, l_2016])

In [ ]:
new_2017 = pd.DataFrame()
for sector in list(set(prob_sec_mean_2017.Sector)):
    l_2017 = comp_test_hard_2017[comp_test_hard_2017.Sector == sector]
    l_2017 = l_2017.sort_values(by = [ 'Prob_1'], ascending = False).head(int(prob_sec_mean_2017['num'][prob_sec_mean_2017.Sector == sector]))
    new_2017 = pd.concat([new_2017, l_2017])

In [ ]:
itog_2016 = new_2016.sort_values(by = [ 'Prob_1','Sector'], ascending = False)
itog_2017 = new_2017.sort_values(by = [ 'Prob_1','Sector'], ascending = False)

In [ ]:
itog_2016.to_excel('proba_2016.xlsx')
itog_2017.to_excel('proba_2017.xlsx')